In [ ]:
# Global Setup!
isWeighted = False
nTrees     = 100
maxDepth   = 2

filtered      = False
plotVariables = True

version  = "All"
removals = {*range(0,92),*range(93,95)}

#version  = "Original"
#removals = {*range(7,21),*range(26,55),*range(61,90),*range(91,95)}

#version  = "NewEventShapes"
#removals = {*range(2,4),*range(7,9),*range(11,21),*range(26,55),
#            *range(61,90),*range(91,95)}

#version  = "Smooth"
#removals = {*range(7,20),*range(22,25),*range(26,30),*range(32,52),
#            *range(55,58),*range(59,63),*range(65,85)}

#version  = "EnergyVars"
#removals = {*range(7,20),*range(25,32),*range(33,38),*range(39,45),
#            *range(46,50),*range(51,52),*range(58,65),*range(66,71),
#            *range(72,78),*range(79,83),*range(84,85)}

#version  = "NewEventVars"
#removals = {*range(7,16),*range(17,18),*range(25,52),*range(58,85)}

#version  = "Basic"
#removals = {*range(1,21),*range(24,55),*range(59,90),*range(91,95)}

#version  = "BasicPlus"
#removals = {*range(2,4),*range(5,6),*range(7,21),*range(25,55),
#            *range(60,90),*range(91,95)}

#version  = "BasicAndEnergy"
#removals = {*range(1,20),*range(23,32),*range(33,45),*range(46,50),
#            *range(51,52),*range(56,65),*range(66,78),*range(79,83),
#            *range(84,85)}

#version  = "BasicPlusAndEnergy"
#removals = {*range(2,4),*range(5,6), *range(7,20),*range(24,32),
#            *range(33,45),*range(46,50),*range(51,52),*range(57,65),
#            *range(66,78),*range(79,83),*range(84,85)}

#version  = "EnergyVarsSmoothdEdx"
#removals = {*range(7,20),*range(25,33),*range(34,38),*range(39,45),
#            *range(46,50),*range(51,52),*range(58,66),*range(67,71),
#            *range(72,78),*range(79,83),*range(84,85)}

#version  = "Linking"
#removals = {*range(7,21),*range(26,55),*range(61,90)}

#version  = "Optimum1"
#removals = {*range(2,4),*range(5,6),*range(7,21),*range(25,33),
#            *range(34,45),*range(46,55),*range(60,68),*range(69,80),
#            *range(81,90),*range(93,94)}

#version  = "Optimum2"
#removals = {*range(2,4),*range(5,6),*range(7,21),*range(25,33),
#            *range(34,45),*range(46,54),*range(60,68),*range(69,80),
#            *range(81,89),*range(93,94)}

#version  = "OptimumSmooth"
#removals = {*range(2,4),*range(5,6),*range(7,21),*range(23,26),
#            *range(27,31),*range(32,34),*range(35,45),*range(46,55),
#            *range(58,61),*range(62,66),*range(67,69),*range(70,80),
#            *range(81,90),*range(93,94)}

#version  = "OptimumMC"
#removals = {*range(2,4),*range(5,6),*range(7,21),*range(23,24),
#            *range(25,29),*range(30,37),*range(38,45),*range(46,55),
#            *range(58,59),*range(60,64),*range(65,72),*range(73,80),
#            *range(81,90),*range(93,94)}

#version  = "OptimumStripped"
#removals = {*range(0,6),*range(7,21),*range(25,33),*range(34,45),
#            *range(46,55),*range(60,68),*range(69,80),*range(81,90),
#            *range(93,95)}

#version  = "OptimumStrippedPlusSharedEnergy"
#removals = {*range(0,6),*range(7,21),*range(25,33),*range(34,45),
#            *range(46,55),*range(60,68),*range(69,80),*range(81,90),
#            *range(94,95)}

#version  = "OptimumMinusLinking"
#removals = {*range(2,4),*range(5,6),*range(7,21),*range(25,33),
#            *range(34,45),*range(46,55),*range(60,68),*range(69,80),
#            *range(81,90),*range(91,95)}

#version  = "OptimumStrippedSmoothdEdx"
#removals = {*range(0,6),*range(7,21),*range(25,34),*range(35,45),
#            *range(46,55),*range(60,69),*range(70,80),*range(81,90),
#            *range(93,95)}

#version  = "NoBeamDW"
#removals = {*range(7,22),*range(26,55),*range(56,57),*range(61,90),*range(91,95)}

#version  = "Final"
#removals = {*range(2,4),*range(7,9),*range(11,21),*range(26,33),
#            *range(34,45),*range(46,55),*range(61,68),*range(69,80),
#            *range(81,90),*range(93,95)}

if isWeighted: version += "_Weighted"
if filtered  : version += "_Filtered"
version += ("_" + str(nTrees) + "_" + str(maxDepth))

print(version)

In [ ]:
# Setup notifications
from time import time as tme
from IPython import get_ipython
from IPython.display import Audio, display

class Beeper:

    def __init__(self, threshold, **audio_kwargs):
        self.threshold = threshold
        self.start_time = None    # time in sec, or None
        self.audio = audio_kwargs

    def pre_execute(self):
        if not self.start_time:
            self.start_time = tme()

    def post_execute(self):
        end_time = tme()
        if self.start_time and end_time - self.start_time > self.threshold:
            audio = Audio(**self.audio, autoplay=True)
            display(audio)
        self.start_time = None

beeper = Beeper(5, filename='/usr/share/sounds/gnome/default/alerts/drip.ogg')

ipython = get_ipython()
ipython.events.register('pre_execute', beeper.pre_execute)
ipython.events.register('post_execute', beeper.post_execute)

In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add path for LArMachineLearningData
import sys
pandoraMVADir = os.environ['PANDORAMVADIR']
dataDir       = os.environ['PANDORABDTDATADIR']

sys.path.append(pandoraMVADir + 'LArMachineLearningData/scripts')

# Import pandora libraries
from importlib import reload
from PandoraBDT import *

# Import concatenation tool
from itertools import chain

# Import relevant SKLearn libraries
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn import metrics

# Set global params
testTrainFraction = 0.5
nCores = -1

In [ ]:
# Data and Name
trainingFile = dataDir + 'training_file_vertex_mix_new.txt'
BDTName = "VertexVertex"

# Directories
plotsDir = pandoraMVADir + 'bdt/plots/' + BDTName + '/' + version + '/'
saveDir  = pandoraMVADir + 'bdt/trained/' + BDTName + '/' + version + '/'
print(plotsDir)
print(saveDir)

if not os.path.exists(plotsDir):
    os.makedirs(plotsDir)
    
if not os.path.exists(saveDir):
    os.makedirs(saveDir)

In [ ]:
# Set some analysis specific things
allFeatureNames = ['Event_Showeryness', # 0
                   'Event_Energy',
                   'Event_Volume',
                   'Event_Longitudinality',
                   'Event_Num_Hits', 
                   'Event_Num_Clusters', # 5
                   'Event_Num_Candidates', 
                   
                   'Event_Area', # 7
                   'Event_Longitudinality2D',
                   'Event_Area_Average_Z',
                   'Event_Longitudinality2D_Average_Z', #10
                   'Event_Hit_Proportion',
                   'Event_Energy_Proportion',
                   'Event_Hit_Outlier_Proportion',
                   'Event_Energy_Outlier_Proportion',
                   'Event_Hit_Harsh_Proportion', # 15
                   'Event_Energy_Harsh_Proportion',
                   'Event_Hit_Outlier_Harsh_Proportion',
                   'Event_Energy_Outlier_Harsh_Proportion',
                   'Event_Num_Regions',
                   'Event_Num_Region_Candidates', # 20

                   'Vertex1_Beam_Deweighting', # 21
                   'Vertex1_Energy_Kick',
                   'Vertex1_Global_Asymmetry', 
                   'Vertex1_Local_Asymmetry',
                   'Vertex1_Shower_Asymmetry', # 25

                   'Vertex1_Global_Smooth_Asymmetry', # 26
                   'Vertex1_Global_Asymmetry_AC',
                   'Vertex1_Global_Smooth_Asymmetry_AC',
                   'Vertex1_Global_Asymmetry_MC',
                   'Vertex1_Global_Smooth_Asymmetry_MC', # 30
                   'Vertex1_Local_Smooth_Asymmetry',
                   'Vertex1_Shower_Smooth_Asymmetry',
                   'Vertex1_dEdx_Asymmetry',
                   'Vertex1_dEdx_Smooth_Asymmetry',
                   'Vertex1_dEdx_Asymmetry_AC', # 35
                   'Vertex1_dEdx_Smooth_Asymmetry_AC',
                   'Vertex1_dEdx_Asymmetry_MC',
                   'Vertex1_dEdx_Smooth_Asymmetry_MC',
                   'Vertex1_dEdx_Deviation',
                   'Vertex1_dEdx_Deviation_AC', # 40
                   'Vertex1_dEdx_Deviation_MC',
                   'Vertex1_Braggness',
                   'Vertex1_Braggness_AC',
                   'Vertex1_Braggness_MC',
                   'Vertex1_Energy', # 45
                   'Vertex1_Average_Energy',
                   'Vertex1_Charge',
                   'Vertex1_Average_Charge',
                   'Vertex1_Energy_Ratio',
                   'Vertex1_Charge_Ratio', # 50
                   'Vertex1_Num_Local_Clusters',
                   'Vertex1_Num_Local_Sliding_Fits',
                   'Vertex1_Fit_Cluster_Ratio',
                   'Vertex1_Mean_Scatter',
                   'Vertex1_rPhi', # 55

                   'Vertex2_Beam_Deweighting', # 56
                   'Vertex2_Energy_Kick',
                   'Vertex2_Global_Asymmetry',
                   'Vertex2_Local_Asymmetry',
                   'Vertex2_Shower_Asymmetry', # 60

                   'Vertex2_Global_Smooth_Asymmetry', # 61
                   'Vertex2_Global_Asymmetry_AC',
                   'Vertex2_Global_Smooth_Asymmetry_AC',
                   'Vertex2_Global_Asymmetry_MC',
                   'Vertex2_Global_Smooth_Asymmetry_MC', # 65
                   'Vertex2_Local_Smooth_Asymmetry',
                   'Vertex2_Shower_Smooth_Asymmetry',
                   'Vertex2_dEdx_Asymmetry',
                   'Vertex2_dEdx_Smooth_Asymmetry',
                   'Vertex2_dEdx_Asymmetry_AC', # 70
                   'Vertex2_dEdx_Smooth_Asymmetry_AC',
                   'Vertex2_dEdx_Asymmetry_MC',
                   'Vertex2_dEdx_Smooth_Asymmetry_MC',
                   'Vertex2_dEdx_Deviation',
                   'Vertex2_dEdx_Deviation_AC', # 75
                   'Vertex2_dEdx_Deviation_MC',
                   'Vertex2_Braggness',
                   'Vertex2_Braggness_AC',
                   'Vertex2_Braggness_MC',
                   'Vertex2_Energy', # 80
                   'Vertex2_Average_Energy',
                   'Vertex2_Charge',
                   'Vertex2_Average_Charge',
                   'Vertex2_Energy_Ratio',
                   'Vertex2_Charge_Ratio', # 85
                   'Vertex2_Num_Local_Clusters',
                   'Vertex2_Num_Local_Sliding_Fits',
                   'Vertex2_Fit_Cluster_Ratio',
                   'Vertex2_Mean_Scatter',
                   'Vertex2_rPhi', # 90
                   
                   'Shared_Separation', # 91
                   'Shared_Axis_Hits',
                   'Shared_Axis_Energy',
                   'Shared_Axis_Ratio'
                  ]

# Set background and signal label names
params = {
    'labelNames': ['Incorrect Vertex','Correct Vertex'],
    'signalDefs': [0, 1],
    'signalCols': ['r', 'b']
}

# Create the base BDT to vary the params from and compare to
baseBDT = AdaBoostClassifier(DecisionTreeClassifier(max_depth=maxDepth),algorithm='SAMME', 
                             random_state=42, n_estimators=nTrees)

In [ ]:
# Load the data
allData, nFeatures, nExamples = LoadData(trainingFile, ',')

# Take weights before removing variables
weightsOrg = GetWeights(allData,20)

# Get vetos before removing variables
vetosOrg   = GetVetos(allData,91,0.3)

# Remove unwanted features
data         = RemoveFeature(allData,removals)
featureNames = []

for i in [x for x in range(0,nFeatures) if x not in removals]:
    featureNames.append(allFeatureNames[i])

del allData, allFeatureNames

# Redefine size
nFeatures = len(data[0]) - 1

# Check removals
print(nFeatures, " =? ", len(featureNames))
print(featureNames)

In [ ]:
featuresOrg, labelsOrg           = SplitTrainingSet(data, nFeatures)
features, labels, weights, vetos = Randomize(featuresOrg, labelsOrg, weightsOrg, vetosOrg, True)

# Split into train and test samples
xUnfTrain, yUnfTrain, weightsUnfTrain, vetosTrain, xTest, yTest, weightsTest = Sample(features, labels, weights, vetos, testTrainFraction)

if filtered :
    xTrain, yTrain, weightsTrain = Filter(xUnfTrain,yUnfTrain,weightsUnfTrain, vetosTrain)
else :
    xTrain       = xUnfTrain
    yTrain       = yUnfTrain
    weightsTrain = weightsUnfTrain
    
# Split into signal and background based on the true labels
signalFeatures = features[labels==1]
backgroundFeatures = features[labels==0]

# Check the features array is the same size as the feature names array
print('Number of features: ' + str(len(featureNames)))
print('Shape of data array: ' + str(np.shape(features)))
print('Total: ' + str(len(features)) + ', signal: ' +
      str(len(signalFeatures)) + ' and background: ' +
      str(len(backgroundFeatures)))
print(len(weights))
print(len(vetos))
print(str(len(xTrain)) + ' vs. ' + str(len(xTest)))

In [ ]:
# Construct the Pandas dataframe
# First create a dictionary
allDict = {featureNames[i]: features[:, i] for i in range(nFeatures)}
allDict.update({'Labels': labels})

# Create the Pandas dataframe, create seperate df for signal/background
df = pd.DataFrame(data=allDict)

In [ ]:
# Make plots drawing the variables for signal/background
if plotVariables : DrawVariablesDF(df, params, plotsDir, save = True)

In [ ]:
# Make correlation matricies
if plotVariables :
    Correlation(df[df['Labels']==params['signalDefs'][0]], params['labelNames'][0] + ' Correlation Matrix',plotsDir, save = True)
    Correlation(df[df['Labels']==params['signalDefs'][1]], params['labelNames'][1] + ' Correlation Matrix',plotsDir, save = True)

In [ ]:
# Reference BDT with controlled hyperparams
if isWeighted : baseBDT.fit(xTrain,yTrain, sample_weight = weightsTrain)
else : baseBDT.fit(xTrain,yTrain)

In [ ]:
# Plot ROC curves
fig, ax = plt.subplots()
metrics.plot_roc_curve(baseBDT, xTest, yTest, ax=ax)

plt.title("ROC Curves")
ax.invert_xaxis()
ax.legend()
ax.grid()

plt.savefig(plotsDir + '/' + "roc.png", bbox_inches='tight')
plt.savefig(plotsDir + '/' + "roc.pdf", bbox_inches='tight')

In [ ]:
# Plot Confusion Matricies
fig, ax = plt.subplots()
metrics.plot_confusion_matrix(baseBDT, xTest, yTest, display_labels=params['labelNames'],
                             ax=ax, normalize='true')
ax.invert_xaxis()
plt.title("Confusion matrix (True Normalised)")

plt.savefig(plotsDir + '/' + "confusion_matrix.png", bbox_inches='tight')
plt.savefig(plotsDir + '/' + "confusion_matrix.pdf", bbox_inches='tight')
plt.show()

In [ ]:
# Print more detailed performance info
bdtPredicted = baseBDT.predict(xTest)

print ("Background (0): ", params['labelNames'][0])
print ("Signal (1): ", params['labelNames'][1])
print ("BDT:\n", metrics.classification_report(yTest, bdtPredicted))

In [ ]:
# Plot importance of features
importanceDF = pd.DataFrame({'Features': featureNames, 'Importance Score':baseBDT.feature_importances_})
print (importanceDF.sort_values(by=['Importance Score']))
ax = importanceDF.sort_values(by=['Importance Score'])\
    .plot(kind='barh', x='Features', y='Importance Score')

plt.savefig(plotsDir + '/' + "feature_importance.png", bbox_inches='tight')
plt.savefig(plotsDir + '/' + "feature_importance.pdf", bbox_inches='tight')

In [ ]:
import PandoraBDT
reload (PandoraBDT)
from PandoraBDT import *

parameters = {
  'ClassNames': ['True Vertex', 'Incorrect Vertex'],
  'SignalDefinition': [1, 0],
  'PlotColors': ['b', 'r'],
  'nBins': 100,
  'PlotStep': 1.0,
  'OptimalBinCut': 0,
  'OptimalScoreCut': 0.0,
  'nTrees': 100,
  'TreeDepth': 3
}

FindOptimalSignificanceCut(baseBDT, xTest, yTest, parameters)
if isWeighted : PlotBdtScoresWeight(baseBDT, xTest, yTest, weightsTest, xTrain, yTrain, 'Vertex Vertex ' + version, parameters, plotsDir, save=True)
else : PlotBdtScores(baseBDT, xTest, yTest, xTrain, yTrain, 'Vertex Vertex ' + version, parameters, plotsDir, save=True)

In [ ]:
WriteXmlFile(saveDir + BDTName + ".xml", baseBDT, BDTName)
SerializeToPkl(saveDir + BDTName + ".pkl", baseBDT)